In [1]:
"""
Ce script combine les tokens du tiers Combined pour reformer le tier trans.
"""

import os
import shutil
import subprocess

import textgrid
from praatio import tgio
from conll3 import *

In [2]:
def split_long_sentence(sentence: str, max_length:int=150) -> list:
    """
    Divise une phrase en plusieurs phrases de longueur maximale max_length. Pour que SPPAS puisse faire la tokenisation et autres analyses.

    Parameters:
    sentence (str): la phrase à diviser
    max_length (int): la longueur maximale des phrases

    Returns:
    sentences (list): la liste des phrases divisées

    Variables:
    words (list): la liste des mots de la phrase
    sentences (list): la liste des phrases divisées
    current_sentence (str): la phrase en cours de construction
    """
    words = sentence.split()
    sentences = []
    current_sentence = ""
    # print("sentences before :", sentence)
    
    for word in words:
        if len(current_sentence) + len(word) + 1 <= max_length:
            current_sentence += (word + " ")
        else:
            sentences.append(current_sentence.strip())
            current_sentence = word + " "
    
    if current_sentence:
        sentences.append(current_sentence.strip())
    
    # print("sentences after :", sentences)
    return sentences

def create_new_trans(textgrid_file: str, new: str):
    """
    Combine les tokens du tiers Combined pour reformer le tier trans en assurant
    que chaque phrase ne dépasse pas 150 caractères et en corrigeant les intervalles.

    Parameters:
    textgrid_file (str): le chemin vers le fichier TextGrid
    new (str): le nom du nouveau fichier TextGrid

    Returns:
    None

    Variables:
    tg (TextGrid): le fichier TextGrid à ouvrir
    combined_tier (list): la liste des intervalles du tiers
    trans (list): la liste des intervalles du nouveau tiers
    new_sentence (str): la phrase en cours de construction
    start_sentence (float): le début de la phrase en cours de construction
    end_sentence (float): la fin de la phrase en cours de construction
    """    
    tg = tgio.openTextgrid(textgrid_file)
    combined_tier = tg.tierDict["Combined"].entryList

    trans = []
    new_sentence = ""
    start_sentence = None
    end_sentence = None

    for interval in combined_tier:
        start, end, label = interval
        if label != "#":
            if start_sentence is None:
                start_sentence = start
            end_sentence = end
            new_sentence += label + " "
        else:
            if new_sentence:
                # print("\n new_sentence :", new_sentence)
                # Split long sentences before adding to trans
                sentences = split_long_sentence(new_sentence.strip())
                # print("sentences trans:", sentences)
                # Calculate new intervals for each split sentence
                current_start = start_sentence
                for s in sentences:
                    current_end = current_start + (end_sentence - start_sentence) * (len(s.split()) / len(new_sentence.strip().split()))
                    trans.append([current_start, current_end, s])
                    current_start = current_end
                new_sentence = ""
                start_sentence = None
                end_sentence = None
            trans.append([start, end, "#"])

    # Ajouter la dernière phrase si elle n'est pas suivie par un #
    if new_sentence:
        # print("new_sentence :", new_sentence)
        sentences = split_long_sentence(new_sentence.strip())
        current_start = start_sentence
        for s in sentences:
            current_end = current_start + (end_sentence - start_sentence) * (len(s.split()) / len(new_sentence.strip().split()))
            trans.append([current_start, current_end, s])
            current_start = current_end

    # print(trans)
    new_tg = tgio.Textgrid()
    new_tg.addTier(tgio.IntervalTier("trans", trans, tg.minTimestamp, tg.maxTimestamp))

    new_tg.save(new)


In [3]:
def save_textgrid_praat(textgrid_dir):
    """
    Sauvegarder via praat pour avoir le bon format de fichier TextGrid

    Parameters:
    None

    Returns:
    None

    Variables:
    praat_executable (str): le chemin vers l'exécutable de Praat
    praat_script (str): le chemin vers le script Praat
    """
    praat_executable = "/Applications/Praat.app/Contents/MacOS/Praat"
    praat_script = "/Users/perrine/Desktop/Stage_2023-2024/Praat/script/new_save.praat"

    try:
        print("Exécution de Praat pour sauvegarder les fichiers TextGrid")
        result = subprocess.run(
            [praat_executable, "--run", praat_script],
            check=True,
            capture_output=True,
            text=True,
        )
        print("Sortie de Praat : Success")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de l'exécution du script Praat : {e}")
        print("Code de retour :", e.returncode)
        print("Sortie standard :", e.stdout)
        print("Erreur standard :", e.stderr)

In [4]:
def generate_tiers_selection_gold_non_gold_silence(directory: str, directory_gold: str) -> tuple:
    """
    Génère une sélection de tiers à partir de fichiers TextGrid gold et non gold dans un répertoire spécifique.

    Parameters:
    directory (str): Chemin vers le répertoire contenant les fichiers TextGrid non gold.
    directory_gold (str): Chemin vers le répertoire contenant les fichiers TextGrid gold.

    Returns:
    tuple: Deux dictionnaires - un pour les tiers obtenus dans des fichiers gold et un autre pour les tiers obtenus dans les fichiers non-gold.
    """
    tiers_gold = {}
    tiers_non_gold = {}
    for file in sorted(os.listdir(directory)):
        if file.endswith(".TextGrid"):
            if "MG-syll.TextGrid" in file:
                tiers_non_gold[file] = ["SyllAlign"]
            elif "MG-palign.TextGrid" in file:
                tiers_non_gold[file] = ["TokensAlign"]
            elif not any(
                substring in file
                for substring in [
                    "MG-phon.TextGrid",
                    "MG-token.TextGrid",
                    "merged",
                    "MG-syll.TextGrid",
                    "MG-id.TextGrid",
                    "MG-syl_tok.TextGrid",
                    "MG-ipus.TextGrid",
                    "MG-new_ipus.TextGrid",
                    "MG-palign.TextGrid",
                ]
            ):
                tiers_non_gold[file] = ["trans"]

    for file in sorted(os.listdir(directory_gold)):
        if file.endswith(".TextGrid"):
            if "MG-id.TextGrid" in file:
                tiers_gold[file] = ["index"]
            elif not any(
                substring in file
                for substring in [
                    "MG-phon.TextGrid",
                    "MG-token.TextGrid",
                    "merged",
                    "MG-syll.TextGrid",
                    "MG-id.TextGrid",
                    "MG-syl_tok.TextGrid",
                    "MG-ipus.TextGrid",
                    "MG-new_ipus.TextGrid",
                    "MG-palign.TextGrid",
                ]
            ):
                tiers_gold[file] = ["trans"]

    return tiers_gold, tiers_non_gold


def merge_gold_non_gold(directory: str, directory_gold: str, tiers_gold: dict, tiers_non_gold: dict, base_name_file: str, merged: str = None) -> None:
    """
    Fusionne les différents tiers en un seul fichier TextGrid.

    Parameters:
    directory (str): Chemin vers le répertoire contenant les fichiers TextGrid non gold.
    directory_gold (str): Chemin vers le répertoire contenant les fichiers TextGrid gold.
    tiers_gold (dict): Dictionnaire des tiers obtenus dans des fichiers gold.
    tiers_non_gold (dict): Dictionnaire des tiers obtenus dans des fichiers non-gold.
    base_name_file (str): Nom pour le fichier de sortie.
    merged (str): Dossier de destination pour le fichier fusionné (optionnel).

    Returns:
    None
    """
    merged_textgrid = textgrid.TextGrid()

    tier_order = ["trans", "index", "trans", "TokensAlign", "SyllAlign"]
    added_tiers = set()

    for tier_name in tier_order:
        if tier_name == "trans":
            # Ajout du premier "trans" des fichiers gold
            for file, tiers_list in tiers_gold.items():
                directory_used = directory_gold
                for tier in tiers_list:
                    if tier_name not in added_tiers:
                        print(f"Adding tier '{tier_name}' from file: {directory_used}, {file}")
                        tg = textgrid.TextGrid.fromFile(os.path.join(directory_used, file))
                        current_tier = tg.getFirst(tier_name)
                        if current_tier is not None:
                            merged_textgrid.append(current_tier)
                            added_tiers.add(tier_name)
        else:
            # Ajout des autres tiers des fichiers non gold
            for file, tiers_list in tiers_non_gold.items():
                directory_used = directory
                for tier in tiers_list:
                    if tier_name not in added_tiers:
                        print(f"Adding tier '{tier_name}' from file: {directory_used}, {file}")
                        tg = textgrid.TextGrid.fromFile(os.path.join(directory_used, file))
                        current_tier = tg.getFirst(tier_name)
                        if current_tier is not None:
                            merged_textgrid.append(current_tier)
                            added_tiers.add(tier_name)

    # Ajout du deuxième "trans" des fichiers non gold
    for file, tiers_list in tiers_non_gold.items():
        directory_used = directory
        for tier in tiers_list:
            if tier == "trans":
                print(f"Adding tier '{tier}' from file: {directory_used}, {file}")
                tg = textgrid.TextGrid.fromFile(os.path.join(directory_used, file))
                current_tier = tg.getFirst(tier)
                if current_tier is not None:
                    current_tier.name = "transAuto"
                    merged_textgrid.append(current_tier)

    if merged:
        merged_textgrid.write(os.path.join(merged, f"{base_name_file}-merged.TextGrid"))
    else:
        merged_textgrid.write(os.path.join(directory, f"{base_name_file}-merged.TextGrid"))


In [5]:
def sppas_tokenisation_syllabification(new_textgrid: str, wave_file: str):
    """
    Tokenisation et syllabification des intervalles du tiers
    """
    command = f"python3 /Users/perrine/Desktop/Stage_2023-2024/SPPAS-4/sppas/bin/annotation.py -I {wave_file} -I {new_textgrid} -l pcm -e .TextGrid --textnorm --phonetize --alignment --syllabify"
    subprocess.run(command, shell=True)


# textgrid_dir = "/Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_nongold_sppas/"
# textgrid_dir_gold = (
#     "/Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_nongold_sppas/")

# textgrid_dir = "/Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_nongold/"
# textgrid_dir_gold = (
#     "/Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_nongold/")

textgrid_dir = "/Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/"
textgrid_dir_gold = "/Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV/"

In [6]:
for root, dirs, files in os.walk(textgrid_dir):
    for file in files:
        if file.endswith("-syl_tok.TextGrid"):
            print(file)
            textgrid_file = os.path.join(root, file)
            output_dir = os.path.join(root, "new")
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            new_textgrid = os.path.join(
                output_dir, file.replace("-syl_tok.TextGrid", "-new.TextGrid")
            )
            create_new_trans(textgrid_file, new_textgrid)

            rename_new_textgrid = os.path.join(
                output_dir, new_textgrid.replace("-new.TextGrid", ".TextGrid")
            )
            os.rename(new_textgrid, rename_new_textgrid)
            wave_file = os.path.join(root, file.replace("-syl_tok.TextGrid", ".wav"))

            # Copier le fichier WAV dans le répertoire 'new'
            new_wave_file = os.path.join(output_dir, os.path.basename(wave_file))
            if os.path.exists(wave_file):
                shutil.copy(wave_file, new_wave_file)
            else:
                print(f"Fichier WAV non trouvé pour {file}")



IBA_32_Tori-By-Samuel_MG-syl_tok.TextGrid
WAZA_10_Bluetooth-Lifestory_MG-syl_tok.TextGrid
IBA_04_Alaska-Pepe_MG-syl_tok.TextGrid
IBA_03_Womanisers_MG-syl_tok.TextGrid
PRT_07_Drummer_MG-syl_tok.TextGrid
LAG_21_I-Like-Stout_MG-syl_tok.TextGrid
KAD_13_Entering-University_MG-syl_tok.TextGrid
IBA_02_Igwe-Festival_MG-syl_tok.TextGrid
PRT_01_Banga-Soup_MG-syl_tok.TextGrid
IBA_33_News-Comments_MG-syl_tok.TextGrid
IBA_34_News-Report-By-Samuel_MG-syl_tok.TextGrid
LAG_11_Adeniyi-Lifestory_MG-syl_tok.TextGrid
ABJ_INF_12_Evictions_MG-syl_tok.TextGrid
KAD_12_Mechanic-At-Work_MG-syl_tok.TextGrid
KAD_15_Money-Wahala_MG-syl_tok.TextGrid
LAG_27_Shawarma_MG-syl_tok.TextGrid
BEN_34_Tale_MG-syl_tok.TextGrid
BEN_02_Andrew-Lifestory_MG-syl_tok.TextGrid
JOS_20_Beauty-Of-Jos_MG-syl_tok.TextGrid
WAZL_15_MC-Abi_MG-syl_tok.TextGrid
ABJ_GWA_03_Cost-Of-Living-In-Abuja_MG-syl_tok.TextGrid
JOS_19_Bukuru_MG-syl_tok.TextGrid
ABJ_GWA_10_Steven-Lifestory_MG-syl_tok.TextGrid
ONI_10_Sport-Commentary_MG-syl_tok.TextGrid
ONI

In [7]:
# Sauvegarder le fichier TextGrid via Praat
save_textgrid_praat(textgrid_dir)


Exécution de Praat pour sauvegarder les fichiers TextGrid
Sortie de Praat : Success



In [8]:
# Tokenisation et syllabification des intervalles du tiers
for root, dirs, files in os.walk(textgrid_dir):
    for dir_name in dirs:
        if dir_name != ".DS_Store":
            subdir_path = os.path.join(root, dir_name, "new")
            if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
                for file in os.listdir(subdir_path):
                    if file.endswith("_M.TextGrid") or file.endswith("_MG.TextGrid"):
                        textgrid_file = os.path.join(subdir_path, file)
                        wave_file = os.path.join(
                            subdir_path, file.replace(".TextGrid", ".wav")
                        )
                        if os.path.exists(wave_file):
                            print(
                                f"Tokenisation et syllabification de {textgrid_file}, {wave_file}\n"
                            )
                            sppas_tokenisation_syllabification(textgrid_file, wave_file)
                        else:
                            print(f"Fichier WAV non trouvé pour {textgrid_file}\n")


Tokenisation et syllabification de /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new/IBA_32_Tori-By-Samuel_MG.TextGrid, /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new/IBA_32_Tori-By-Samuel_MG.wav



2024-06-10 18:10:01,905 [INFO] Logging redirected to StreamHandler (level=15).
2024-06-10 18:10:01,905 [INFO] Check directories and create if not existing:
2024-06-10 18:10:01,905 [INFO]  - Write access to SPPAS package directory is granted.
2024-06-10 18:10:01,905 [INFO]  - The folder for logs is OK.
2024-06-10 18:10:01,905 [INFO]  - The folder for the workspaces is OK.
2024-06-10 18:10:01,905 [INFO]  - The trash is OK.
2024-06-10 18:10:02,022 [WARNING] Support of images is disabled because it requires image feature.
2024-06-10 18:10:02,023 [WARNING] Support of videos is disabled because it requires video feature.
2024-06-10 18:10:02,067 [WARNING] Face detection is disabled because it requires video feature.
2024-06-10 18:10:02,073 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-06-10 18:10:02,075 [WARNING] Person face identification is disabled because it requires the 'video' feature.
2024-06-10 18:10:02,079 [INFO] Configuration file /Users/perrine/Desktop/Sta

------------------------------------------------------------------------

SPPAS   -  Version 4.21
Copyright (C) 2011-2024 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new/IBA_32_Tori-By-Samuel_MG.wav
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new/IBA_32_Tori-By-Samuel_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new/IBA_32_Tori-By-Samuel_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

         * * * 

2024-06-10 18:10:07,708 [INFO] Logging redirected to StreamHandler (level=15).
2024-06-10 18:10:07,708 [INFO] Check directories and create if not existing:
2024-06-10 18:10:07,709 [INFO]  - Write access to SPPAS package directory is granted.
2024-06-10 18:10:07,709 [INFO]  - The folder for logs is OK.
2024-06-10 18:10:07,709 [INFO]  - The folder for the workspaces is OK.
2024-06-10 18:10:07,709 [INFO]  - The trash is OK.
2024-06-10 18:10:07,782 [WARNING] Support of images is disabled because it requires image feature.
2024-06-10 18:10:07,783 [WARNING] Support of videos is disabled because it requires video feature.
2024-06-10 18:10:07,829 [WARNING] Face detection is disabled because it requires video feature.
2024-06-10 18:10:07,834 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-06-10 18:10:07,836 [WARNING] Person face identification is disabled because it requires the 'video' feature.
2024-06-10 18:10:07,839 [INFO] Configuration file /Users/perrine/Desktop/Sta

------------------------------------------------------------------------

SPPAS   -  Version 4.21
Copyright (C) 2011-2024 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/WAZA_10/new/WAZA_10_Bluetooth-Lifestory_MG.wav
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/WAZA_10/new/WAZA_10_Bluetooth-Lifestory_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/WAZA_10/new/WAZA_10_Bluetooth-Lifestory_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 perce

...

------------------------------------------------------------------------

SPPAS   -  Version 4.21
Copyright (C) 2011-2024 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/LAG_31/new/LAG_31_Road-Safety_MG.wav
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/LAG_31/new/LAG_31_Road-Safety_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/LAG_31/new/LAG_31_Road-Safety_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

         * * *  Phonetiz

2024-06-10 18:18:25,085 [INFO] Logging redirected to StreamHandler (level=15).
2024-06-10 18:18:25,085 [INFO] Check directories and create if not existing:
2024-06-10 18:18:25,085 [INFO]  - Write access to SPPAS package directory is granted.
2024-06-10 18:18:25,085 [INFO]  - The folder for logs is OK.
2024-06-10 18:18:25,085 [INFO]  - The folder for the workspaces is OK.
2024-06-10 18:18:25,085 [INFO]  - The trash is OK.
2024-06-10 18:18:25,171 [WARNING] Support of images is disabled because it requires image feature.
2024-06-10 18:18:25,171 [WARNING] Support of videos is disabled because it requires video feature.
2024-06-10 18:18:25,207 [WARNING] Face detection is disabled because it requires video feature.
2024-06-10 18:18:25,210 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-06-10 18:18:25,212 [WARNING] Person face identification is disabled because it requires the 'video' feature.
2024-06-10 18:18:25,214 [INFO] Configuration file /Users/perrine/Desktop/Sta

------------------------------------------------------------------------

SPPAS   -  Version 4.21
Copyright (C) 2011-2024 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_40/new/IBA_40_Christ-Passion-Prologue_MG.wav
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_40/new/IBA_40_Christ-Passion-Prologue_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_40/new/IBA_40_Christ-Passion-Prologue_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1

2024-06-10 18:18:27,433 [INFO] Logging redirected to StreamHandler (level=15).
2024-06-10 18:18:27,433 [INFO] Check directories and create if not existing:
2024-06-10 18:18:27,433 [INFO]  - Write access to SPPAS package directory is granted.
2024-06-10 18:18:27,433 [INFO]  - The folder for logs is OK.
2024-06-10 18:18:27,433 [INFO]  - The folder for the workspaces is OK.
2024-06-10 18:18:27,433 [INFO]  - The trash is OK.
2024-06-10 18:18:27,515 [WARNING] Support of images is disabled because it requires image feature.
2024-06-10 18:18:27,515 [WARNING] Support of videos is disabled because it requires video feature.
2024-06-10 18:18:27,552 [WARNING] Face detection is disabled because it requires video feature.
2024-06-10 18:18:27,556 [WARNING] Hand&Pose is disabled because it requires video feature.
2024-06-10 18:18:27,557 [WARNING] Person face identification is disabled because it requires the 'video' feature.
2024-06-10 18:18:27,560 [INFO] Configuration file /Users/perrine/Desktop/Sta

------------------------------------------------------------------------

SPPAS   -  Version 4.21
Copyright (C) 2011-2024 Brigitte Bigi
https://sppas.org/

------------------------------------------------------------------------

Annotations: 
 - textnorm: enabled.
 - phonetize: enabled.
 - alignment: enabled.
 - syllabify: enabled.

Files and folders: 
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/LAG_07/new/LAG_07_Johns-Biography_MG.wav
 - /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/LAG_07/new/LAG_07_Johns-Biography_MG.TextGrid


         * * *  Text Normalization  * * *  
  => Loading resources... (0 percents)
  =>  (0 percents)
  =>  (0 percents)
  => /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/LAG_07/new/LAG_07_Johns-Biography_MG.TextGrid (0 percents)
  => Terminé. (1 / 1 fichiers ont été traités avec succès) (1 percents)

         * *

In [9]:
# Sauvegarder le fichier TextGrid via Praat
save_textgrid_praat(textgrid_dir)

Exécution de Praat pour sauvegarder les fichiers TextGrid
Sortie de Praat : Success



In [10]:
# Fusion des fichiers TextGrid gold et non gold
for root, dirs, files in os.walk(textgrid_dir):
    for dir_name in dirs:
        if dir_name != ".DS_Store":
            subdir_path = os.path.join(root, dir_name, "new")
            # print(subdir_path)
            if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
                non_gold_path = subdir_path
                gold_path = os.path.join(textgrid_dir_gold, dir_name)

                tiers, tiers_combined = generate_tiers_selection_gold_non_gold_silence(
                    non_gold_path, gold_path
                )

                merge_gold_non_gold(
                    non_gold_path,
                    gold_path,
                    tiers,
                    tiers_combined,
                    dir_name,
                    merged=non_gold_path,
                )

# Sauvegarder le fichier TextGrid via Praat
save_textgrid_praat(textgrid_dir)

print("Done")

Adding tier 'trans' from file: /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV/IBA_32, IBA_32_Tori-By-Samuel_MG-id.TextGrid
Adding tier 'trans' from file: /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV/IBA_32, IBA_32_Tori-By-Samuel_MG.TextGrid
Adding tier 'index' from file: /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new, IBA_32_Tori-By-Samuel_MG-palign.TextGrid
Adding tier 'index' from file: /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new, IBA_32_Tori-By-Samuel_MG-syll.TextGrid
Adding tier 'index' from file: /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new, IBA_32_Tori-By-Samuel_MG.TextGrid
Adding tier 'TokensAlign' from file: /Users/perrine/Desktop/Stage_2023-2024/TEXTGRID_WAV_gold_non_gold_TALN_04-05_10ms_webrtcvad3/IBA_32/new, IBA_32_Tori-By-Samuel_MG-palign.TextGrid
Adding tier 'SyllAlign' from file: /Users/